# Recommendation System

In [1]:
import networkx as nx
from armining import *
from io_utils import *
from graphs import *

# Scenario 1 
On suppose ici qu'à des fins d'études statistiques, une chaîne de restauration rapide a enregistré un échantillon de 1000 menus commandés par ses clients sur ses bornes interactives de commande. Cet échantillon est enregistré dans le fichier `dataQuestion1/menus.txt` dont chaque ligne donne un identifiant de la commande suivi de la liste des produits choisis séparés par des virgules. Vous pouvez récupérer ces informations sous forme d'un dictionnaire en utilisant la fonction `read_txt_to_dic()` du module `io_utils`: les clés de ce dictionnaire sont les identifiants de chaque commande, et les valeurs, les listes des produits composant chaque menu commandé.

D'autre part, la chaine de restaurant a mené une enquête anonyme auprès de ses clients pour récolter leur appréciation des produits proposés, sur une échelle de 1 à 5. La note moyenne obtenue par chaque produit est répertoriée dans le fichier `dataQuestion1/notes_moyennes.txt` avec sur chaque ligne, le nom du produit suivi de sa note moyenne.

La chaîne de restaurants souhaiterait proposer à partir de ces informations une recommandation **personnalisée** sur ses bornes interactives de commande. Pour donner un peu de fiabilité aux recommandations et donc ne pas trop agacer les clients avec de mauvaises propositions, on voudrait que la recommandation d'un produit soit correcte au moins une fois sur deux en moyenne, et que l'association du produit recommandé et des produits déjà choisis ait été rencontrée dans au moins 5% des menus enregistrés lors de l'étude statistique. Dans le cas contraire, on estime que la recommandation ne sera pas assez fiable et on préfère ne rien recommander à l'utilisateur sur la borne interactive.

Avec ces contraintes, écrire le code permettant d'afficher le ou les produits que vous recommanderiez sur la borne interactive à un client dans les deux cas suivants:

- l'utilisateur a déjà choisi les produits `Salade` et `Eau`
- l'utilisateur a déjà choisi les produits `Wrap` et `Potatoes`

In [2]:
history = list(map(ItemSet, read_txt_to_dic("dataQuestion1/menus.txt").values()))
support = int(len(history)*.05)
conf = .5
for ant1, ant2 in [("Salade", "Eau"), ("Wrap", "Potatoes")]:
    print(f"{'='*75}\nRecommendation for {ant1} and {ant2} with support of {support} and {conf} confidence:")
    ar = lambda x:ant1 in x.antecedent and ant2 in x.antecedent
    recom = list(filter(ar, mineAssociationRules(history, support, conf)))
    if len(recom): print(*recom)
    else: print("No recommendation")

Recommendation for Salade and Eau with support of 50 and 0.5 confidence:
{'Eau', 'Salade'} -> {'Nuggets'}
Recommendation for Wrap and Potatoes with support of 50 and 0.5 confidence:
No recommendation


# Scenario 2
Des utilisateurs identifiés ont attribué un certain nombre d'étoiles (représentant leur appréciation) à des objets (identifiés par `P001`, `P002`, etc., ici) sur une plateforme de vente en ligne de type CDiscount. Ces appréciations sont contenues dans le fichier `dataQuestion2/etoiles.csv` qui peut être ouvert en utilisant la fonction `read_grades_from_csv()` du module `io_utils`.  
Cette fonction renvoie trois conteneurs:

- un dictionnaire du nombre d'étoiles attribuées, avec comme clés les paires (utilisateur, produit) et comme valeurs les nombres d'étoiles;
- une liste de tous les utilisateurs identifiés;
- une liste de tous les produits.

Écrire le code permettant de prédire les nombres d'étoiles pour les évaluations manquantes avec une prédiction basée utilisateur et d'afficher ces appréciations avec, pour chacune, le nom de l'utilisateur et l'identifiant de l'objet correspondant

In [3]:
given, users, items = read_grades_from_csv("dataQuestion2/etoiles.csv")
empty = [(u,i) for u in users for i in items if (u,i) not in given.keys()]
user_based = lambda x:(x, round(user_based_collaborative_filtering(given, x)))
for (u,i),n in map(user_based, empty):
    print(f'{u:>9} -> {i:>1}: {n}')

  Daniels -> P005: 4
   Holler -> P003: 1
   Holler -> P005: 2
   Holler -> P008: 2
   Holler -> P013: 2
   Litton -> P002: 4
   Litton -> P008: 3
   Litton -> P015: 3
   Mooney -> P003: 2
   Mooney -> P007: 3
   Mooney -> P012: 2
    Smith -> P006: 4
Sternberg -> P006: 4
Sternberg -> P007: 3
Sternberg -> P015: 2
  Whipple -> P004: 3
  Whipple -> P009: 5
  Whipple -> P011: 4
  Whipple -> P012: 3


# Scenario 3

On considère ici que l'entreprise a récolté des données supplémentaires sur son site de vente en ligne concernant ses produits. D'une part elle a pu dégager une liste de paires de produits souvent associés par les utilisateurs, soit parce qu'ils sont fréquemment achetés ensemble, soit parce qu'ils sont souvent consultés tous les deux lors d'une même session de navigation sur le site.  
Cette liste de paires de produits associés est donnée dans le fichier `dataQuestion3/paires_produits.txt`.

D'autre part, l'entreprise a récupéré pour chaque produit le nombre d'avis déposés par les utilisateurs du site depuis leur mise en vente sur le site (on ne connait pas la date de mise en ligne des produits).  
Ces données sont enregistrées dans le fichier `dataQuestion3/produits_avis.txt` avec sur chaque ligne, l'identifiant du produit suivi du nombre d'avis concernant ce produit.  
En outre, on dispose toujours des appréciations attribuées aux produits par des utilisateurs identifiés dans le fichier `dataQuestion3/etoiles.csv`.

## Question 1
Utilisez ces informations pour prédire la note qu'attribuerait l'utilisateur Smith au produit P006 pour une **prédiction basée items** et avec une mesure de similarité qui **n'utilise pas les notes**

In [4]:
graph_1 = nx.readwrite.edgelist.read_edgelist("dataQuestion3/paires_produits.txt", create_using=nx.Graph())
given,_,_ = read_grades_from_csv("dataQuestion3/etoiles.csv")
print(f"Predicted note given by Smith for P006: {item_based_collaborative_filtering(given, ('Smith', 'P006'), graph=graph_1) :.0f}")

Predicted note given by Smith for P006: 4


## Question 2
Donnez des paires de produits non présentes dans le fichier `dataQuestion3/paires_produits.txt` qui pourraient probablement être associés (vous afficherez les 10 paires les plus probables)

In [5]:
for i1,i2,n in top_k_triplets(generic_adamic_adar(graph_1), 10):
    print(f'{i1:>4} -> {i2:>4} with score of {n:.3f}')

P006 -> P001 with score of 1.243
P006 -> P004 with score of 1.243
P006 -> P008 with score of 1.243
P012 -> P003 with score of 1.243
P012 -> P010 with score of 1.243
P012 -> P015 with score of 1.243
P011 -> P009 with score of 0.910
P011 -> P007 with score of 0.910
P011 -> P010 with score of 0.910
P012 -> P014 with score of 0.910


# Scenario 4
On dispose cette fois-ci, en plus des appréciations (fichier `dataQuestion4/etoiles.csv`), d'une hiérarchie de catégories associée à chaque objet. Cette liste est disponible dans le fichier `dataQuestion4/categories_produits.txt`. Par exemple, la première ligne:  
```sh
P001 Livre,BD
```
indique que l'objet `P001` appartient à la catégorie Livre et plus précisément à la sous-catégorie BD. On souhaite pour cette question utiliser une similarité entre objets basée sur ces catégories et non passur les notes.

La fonction `collaborative_filtering()` du module graphs a été modifiée pour pouvoir accepter comme paramètre supplémentaire une liste de catégories associées à des items. Lorsqu'il est utilisé, ce paramètre nommé `categories` doit être un dictionnaire dont les clés sont les identifiants des items, et les valeurs sont les listes de catégories associées. Lorsqu'on veut utiliser le paramètre `categories` il est obligatoire de préciser la fonction de similarité utilisée entre ces listes de catégories en passant sa référence en paramètre de `collaborative_filtering()` (paramètre `similarity_fun`).

La similarité basée catégories entre deux objets sera de 0 s'ils n'appartiennent pas à la même catégorie (et donc pas non plus à la même sous-catégorie), de 0.5 s'ils appartiennent à la même catégorie mais pas à la même sous-catégorie et enfin de 1 s'ils appartiennent à la même catégorie et à la même sous-catégorie.
## Question 1
Construire une fonction qui prend en paramètre deux listes de catégorie+sous-catégorie, et renvoie la mesure de similarité définie ci-dessus. Vous testerez cette fonction avec différentes liste de catégorie+sous-catégorie de votre choix dans le programme.

In [6]:
def similarity(*args):
    sim = int(args[0] == args[1])
    if not sim and args[0][0] == args[1][0]:
        sim = .5
    return sim

In [7]:
categories = read_txt_to_dic("dataQuestion4/categories_produits.txt")
cat_list = list(categories.values())
for i in (3,4,5):
    print(f"{cat_list[0]} and {repr(cat_list[i]):>30} -> {similarity(cat_list[0], cat_list[i])}")

['Livre', 'BD'] and   ['Multimedia', 'Smartphone'] -> 0
['Livre', 'BD'] and                ['Livre', 'BD'] -> 1
['Livre', 'BD'] and   ['Livre', 'Manuel_scolaire'] -> 0.5


## Question 2
Construire une fonction qui permet de calculer une liste de similarités entre paires d'items à partir de catégories associées à ces items. La fonction prend en premier paramètre un dictionnaire où les clés sont des identifiants d'items et les valeurs les listes de catégories/sous-catégories associées, et en second paramètre une liste de tuple d'identifiant d'items (`item1`, `item2`) entre lesquels on souhaite calculer la similarité définie ci-dessus.

Le retour de la fonction doit être de même forme que celui des fonctions `nx.adamic_adar()` et `cosine_sim_all()` : une liste de triplets `(item1, item2, similarity)` où les deux premiers éléments sont les identifiants des items, et le troisième, la mesure de similarité calculée entre ces deux items.

In [8]:
def cat_sim_all(categories, pairs):
    return [
        (u,v,similarity(categories[u], categories[v]))
        for u,v in pairs
    ]

## Question 3
Construire la fonction `item_based_collaborative_filtering_with_categories()` qui permet d'appeler `collaborative_filtering()` du module `graphs` avec un dictionnaire de catégories associées à de sitems et la fonction de similarité de la question précédente, pour de la **recommandation basée items**.

In [9]:
def item_based_collaborative_filtering_with_categories(scores, target, categories, n_neighbors=5, similarity_fun=cat_sim_all, graph=None):
        i_scores = {(item, user): score for ((user, item), score) in scores.items()}
        i_target = (target[1], target[0])
        return collaborative_filtering(i_scores, i_target, n_neighbors, similarity_fun, graph, categories)

## Question 4
Utiliser la fonction `item_based_collaborative_filtering_with_categories()` pour prédire la note qu'attribuerait l'utilisateur `Whipple` au produit `P009`.

In [10]:
given,_,_ = read_grades_from_csv("dataQuestion4/etoiles.csv")
print(f"Predicted note given by Whipple for P009: {item_based_collaborative_filtering_with_categories(given, ('Whipple', 'P009'), categories) :.0f}")

Predicted note given by Whipple for P009: 5
